In [17]:
import torch
# load data and split into train and test sets
from load_data import sim_arr
from sklearn.model_selection import train_test_split

# reshape from batch, height, width, channel, to batch, channel, height, width
sim_arr_transformed = sim_arr.reshape(
    sim_arr.shape[0], sim_arr.shape[3], sim_arr.shape[1], sim_arr.shape[2])
train_set, test_set = train_test_split(
    sim_arr_transformed, test_size=0.2, random_state=42)

# convert to tensor
train_set = torch.tensor(train_set, dtype=torch.float32)
test_set = torch.tensor(test_set, dtype=torch.float32)



In [18]:
from AE_torch import Autoencoder
from search_space import search_space
import optuna
import warnings


warnings.filterwarnings("ignore")


def objective(trial):
    #clear clutter from previous runs
    torch.cuda.empty_cache()

    # define search space
    num_layers, poolsize, channels, kernel_sizes, dilations, activations = search_space(trial, input_dim=3, output_dim=3)

    # define model
    model = Autoencoder(num_layers=num_layers,
                        poolsize=poolsize,
                        channels=channels,
                        kernel_sizes=kernel_sizes,
                        dilations=dilations,
                        activations=activations,
                        epochs=100,
                        batch_size=32,
                        learning_rate=1e-3,
                        data=train_set)
    
    # train model with k-fold cross validation
    val_losses = model.cross_val()
    val_loss = sum(val_losses) / len(val_losses)
    print("Validation loss:", val_loss)
    return val_loss

# delete the study
#optuna.delete_study(study_name="autoencoder_torch1", storage="sqlite:///autoencoder.db")

# define study
study = optuna.create_study(direction="minimize",
                            pruner=optuna.pruners.HyperbandPruner(),
                            study_name="autoencoder_torch3",
                            storage="sqlite:///autoencoder.db",
                            load_if_exists=True)

study.optimize(objective, n_trials=0)

# Get the best hyperparameters
best_params = study.best_params
print("Best hyperparameters:", str(best_params))
print("Best value:", study.best_value)


[I 2024-05-15 06:21:33,333] Using an existing study with name 'autoencoder_torch3' instead of creating a new one.


Best hyperparameters: {'kernel_size_0': 9, 'kernel_size_1': 10, 'kernel_size_2': 13, 'kernel_size_3': 4, 'kernel_size_4': 11, 'dilation_0': 3, 'dilation_1': 2, 'dilation_2': 4, 'dilation_3': 4, 'dilation_4': 2, 'activation_0': 'nn.SELU', 'activation_1': 'nn.Softplus', 'activation_2': 'nn.Tanh', 'activation_3': 'nn.Softplus', 'activation_4': 'nn.Tanh'}
Best value: 0.3946284880628809


In [19]:
# train model with best hyperparameters
num_layers, poolsize, channels, kernel_sizes, dilations, activations = search_space(
    study.best_trial, input_dim=3, output_dim=3)

model = Autoencoder(num_layers=num_layers,
                    poolsize=poolsize,
                    channels=channels,
                    kernel_sizes=kernel_sizes,
                    dilations=dilations,
                    activations=activations,
                    epochs=100,
                    batch_size=32, 
                    learning_rate=1e-3,
                    data=train_set)

#model.cross_val()

In [20]:
"""import numpy as np
import matplotlib.pyplot as plt

i = np.random.randint(0, len(test_set))
in_sim = test_set[i: i + 1]
in_sim = in_sim.to('cuda:0')
out_sim = model(in_sim)
in_sim = in_sim.detach().to('cpu').numpy()
in_sim = in_sim.reshape(in_sim.shape[0], in_sim.shape[2], in_sim.shape[3], in_sim.shape[1])
out_sim = out_sim.detach().to('cpu').numpy()
out_sim = out_sim.reshape(out_sim.shape[0], out_sim.shape[2], out_sim.shape[3], out_sim.shape[1])
fig, ax = plt.subplots(nrows=1, ncols=2)
ax[0].imshow(in_sim[0, ..., 2], vmin=-1, vmax=1, cmap="RdBu")
ax[1].imshow(out_sim[0, ..., 2], vmin=-1, vmax=1, cmap="RdBu")"""

'import numpy as np\nimport matplotlib.pyplot as plt\n\ni = np.random.randint(0, len(test_set))\nin_sim = test_set[i: i + 1]\nin_sim = in_sim.to(\'cuda:0\')\nout_sim = model(in_sim)\nin_sim = in_sim.detach().to(\'cpu\').numpy()\nin_sim = in_sim.reshape(in_sim.shape[0], in_sim.shape[2], in_sim.shape[3], in_sim.shape[1])\nout_sim = out_sim.detach().to(\'cpu\').numpy()\nout_sim = out_sim.reshape(out_sim.shape[0], out_sim.shape[2], out_sim.shape[3], out_sim.shape[1])\nfig, ax = plt.subplots(nrows=1, ncols=2)\nax[0].imshow(in_sim[0, ..., 2], vmin=-1, vmax=1, cmap="RdBu")\nax[1].imshow(out_sim[0, ..., 2], vmin=-1, vmax=1, cmap="RdBu")'

In [21]:
# summarize the model
print(model)

Autoencoder(
  (encoder): Encoder(
    (layers): ModuleList(
      (0): Conv2dSame(3, 11, kernel_size=(9, 9), stride=(1, 1), dilation=(3, 3))
      (1): SELU()
      (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
      (3): Conv2dSame(11, 10, kernel_size=(10, 10), stride=(1, 1), dilation=(2, 2))
      (4): Softplus(beta=1.0, threshold=20.0)
      (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
      (6): Conv2dSame(10, 12, kernel_size=(13, 13), stride=(1, 1), dilation=(4, 4))
      (7): Tanh()
      (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
      (9): Conv2dSame(12, 10, kernel_size=(4, 4), stride=(1, 1), dilation=(4, 4))
      (10): Softplus(beta=1.0, threshold=20.0)
      (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
      (12): Conv2dSame(10, 3, kernel_size=(11, 11), stride=(1, 1), dilation=(2, 2))
      (13): Tanh()
      (14): MaxPool2d(kernel_size=5

In [50]:
from sklearn.model_selection import KFold
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

class KmeansLayer(torch.nn.Module):
    """Kmeans layer that inherits from PyTorch's nn.Module class."""

    def __init__(self, n_clusters):
        super(KmeansLayer, self).__init__()
        self.n_clusters = n_clusters

    def forward(self, feature_array):
        """Forward pass through the K-means layer."""
        # fit K-means model
        kmeans = KMeans(n_clusters=self.n_clusters, random_state=42)
        print(feature_array.shape)
        kmeans.fit(feature_array)

        return kmeans.labels_
    
    def _get_loss(self, feature_array, labels):
        """Calculate the silhouette score."""
        return silhouette_score(feature_array, labels)


class Classifier(torch.nn.Module):
    """Classifier that inherits from PyTorch's nn.Module class."""

    def __init__(self, input_dim, output_dim):
        super(Classifier, self).__init__()
        self.fc = torch.nn.Linear(input_dim, output_dim)

    def forward(self, x):
        """Forward pass through the classifier."""
        x = self.fc(x)
        return x


class JointAutoencoder(torch.nn.Module):
    def __init__(self, encoder, decoder, cluster, full_set):
        super(JointAutoencoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.cluster = cluster
        self.full_set = full_set
        self.epochs = 100
        self.batch_size = 64
        self.data = train_set
        self.optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)

    def forward(self, x):
        """Forward pass through the ClusterAutoencoder."""
        x, indices_list = self.encoder(x)
        with torch.no_grad():
            feature_array, _ = self.encoder(self.full_set)
            features = features.view(features.size(0), -1)
            feature_array = feature_array.cpu().detach().numpy()
        labels = self.cluster(feature_array)
        x = self.decoder(x, indices_list)

        return x, labels

    def _get_reconstruction_loss(self, x, x_hat):
        """Calculate the reconstruction loss."""
        return torch.nn.functional.mse_loss(x_hat, x, reduction='mean')

    def _get_cluster_loss(self, x, cluster_centers):
        """Calculate the clustering loss."""
        pass
    
    def _get_weighted_mse_loss(self, x, cluster_centers):
        pass
        

    def _get_loss(self, x):
        """Calculate the loss function."""
        x_hat, labels = self(x)
        reconstruction_loss = self._get_reconstruction_loss(x, x_hat)
        with torch.no_grad():
            feature_array, _ = self.encoder(self.full_set)
            features = features.view(features.size(0), -1)
            feature_array = feature_array.cpu().detach().numpy()
        cluster_loss = self.cluster._get_loss(feature_array, labels)
        return reconstruction_loss + cluster_loss

    def train_model(self, dataloader, device=torch.device('cuda:0')):
        """Train the autoencoder."""
        self.to(device)
        self.train()

        best_loss = float('inf')
        epochs_no_improve = 0

        for epoch in range(self.epochs):
            running_loss = 0.0

            for batch in dataloader:
                batch = batch.to(device)
                self.optimizer.zero_grad()
                loss = self._get_loss(batch)
                loss.backward()
                self.optimizer.step()
                running_loss += loss.item() * batch.size(0)
            epoch_loss = running_loss / len(dataloader)
            print(f"Epoch {epoch+1}/{self.epochs} Loss: {epoch_loss:.4f}")

            # Early stopping
            if epoch_loss < best_loss:
                best_loss = epoch_loss
                epochs_no_improve = 0
            else:
                epochs_no_improve += 1
                if epochs_no_improve == 5:
                    print(f"Early stopping after {epoch+1} epochs.")
                    break

    def evaluate_model(self, dataloader, device=torch.device('cuda:0')):
        """Evaluate the autoencoder."""
        self.to(device)
        self.eval()
        total_loss = 0.0
        with torch.no_grad():
            for batch in dataloader:
                batch = batch.to(device)
                loss = self._get_loss(batch)
                total_loss += loss.item() * batch.size(0)
        avg_loss = total_loss / len(dataloader)
        print(f"Validation Loss: {avg_loss:.4f}")
        return avg_loss

    def cross_val(self, n_splits=5):
        """Perform cross-validation on the autoencoder."""
        torch.backends.cudnn.benchmark = True
        kf = KFold(n_splits=n_splits, shuffle=True)
        val_losses = []
        for fold, (train_index, val_index) in enumerate(kf.split(self.data)):
            print(f"Fold {fold+1}/{n_splits}")
            train_sampler = torch.utils.data.SubsetRandomSampler(train_index)
            val_sampler = torch.utils.data.SubsetRandomSampler(val_index)

            train_loader = torch.utils.data.DataLoader(
                self.data,
                sampler=train_sampler,
                batch_size=self.batch_size,
                num_workers=12)
            val_loader = torch.utils.data.DataLoader(
                self.data,
                sampler=val_sampler,
                batch_size=self.batch_size,
                num_workers=12)

            self.train_model(train_loader)
            val_loss = self.evaluate_model(val_loader)
            val_losses.append(val_loss)
        return val_losses



# train model with best hyperparameters
num_layers, poolsize, channels, kernel_sizes, dilations, activations = search_space(
    study.best_trial, input_dim=3, output_dim=3)

model = Autoencoder(num_layers=num_layers,
                    poolsize=poolsize,
                    channels=channels,
                    kernel_sizes=kernel_sizes,
                    dilations=dilations,
                    activations=activations,
                    epochs=10,
                    batch_size=32,
                    learning_rate=1e-3,
                    data=train_set)

set = torch.tensor(sim_arr_transformed, dtype=torch.float32)
set = set.to('cuda:0')

model = JointAutoencoder(model.encoder,
                         model.decoder,
                         KmeansLayer(n_clusters=20),
                         set)

model.cross_val()

Fold 1/5
(3, 1, 1)


ValueError: Found array with dim 3. KMeans expected <= 2.

In [41]:
set = torch.tensor(sim_arr_transformed, dtype=torch.float32)
set = set.to('cuda:0')
cluster_centers = model.predict(set)
features = model.encoder(set)

from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=20, init=cluster_centers, n_init=1)
features = features[0]
features = features.view(features.size(0), -1)
features = features.cpu().detach().numpy()
kmeans.fit(features)
k

KMeans(init=array([[0.9999999 , 0.9999999 , 0.9999999 ],
       [0.9999991 , 0.999999  , 0.9999984 ],
       [0.9999991 , 0.999999  , 0.9999984 ],
       [0.99999905, 0.999999  , 0.99999833],
       [0.99999887, 0.9999988 , 0.99999815],
       [0.99999905, 0.999999  , 0.99999833],
       [0.99999905, 0.999999  , 0.99999833],
       [0.99999994, 0.9999999 , 0.9999998 ],
       [0.99999905, 0.999999  , 0.99999833],
       [0.99999905, 0.999999  , 0.99999833],
       [0.9999991 , 0.999999  , 0.9999984 ],
       [0.9999991 , 0.999999  , 0.9999984 ],
       [0.99999905, 0.9999989 , 0.99999833],
       [0.99999905, 0.999999  , 0.9999984 ],
       [0.9999991 , 0.999999  , 0.9999984 ],
       [0.9999991 , 0.999999  , 0.9999984 ],
       [0.99999905, 0.9999989 , 0.99999833],
       [0.9999991 , 0.999999  , 0.9999984 ],
       [0.9999991 , 0.999999  , 0.9999984 ],
       [0.9999991 , 0.999999  , 0.9999984 ]], dtype=float32),
       n_clusters=20, n_init=1)

AttributeError: 'tuple' object has no attribute 'size'